In [1]:
import pandas as pd # biblioteca que permite análise e manipulação de dados
from matminer.datasets import load_dataset # biblioteca que reúne dados de materiais e oferece diferentes ferramentas para manipulá-los

colunas = pd.read_csv("Dados.csv") 
colunas

,0,1,2
0,boltztrap_mp,pf_n,pf_p
1,citrine_thermal_conductivity,k_expt,NaN
2,ucsb_thermoelectrics,kappa [W/mK],NaN


In [2]:
datasets = [load_dataset(dataset) for dataset in colunas["0"]]

In [3]:
coluna_dadaset1 = list(colunas[["1", "2"]].iloc[0]) + ["formula", "structure"]
coluna_dadaset2 = [colunas["1"].iloc[1],"formula"]

In [4]:
datasets[0] = datasets[0][coluna_dadaset1]
datasets[1] = datasets[1][coluna_dadaset2]

In [5]:
dados_juntos = pd.merge(datasets[0], datasets[1], on= "formula")
len(set(dados_juntos["formula"]))

36

In [6]:
from matminer.featurizers.conversions import StrToComposition # composição estequiométrica
from matminer.featurizers.composition import ElementProperty # propriedades dos elementos
# ferramentas oferecidas pelo próprio matminer

# Carregando o dataset
df = datasets[0]

stc = StrToComposition()
df = stc.featurize_dataframe(df, "formula", pbar=False)

# Calculando as propriedades dos elementos
ep_feat = ElementProperty("pymatgen", ["melting_point","thermal_conductivity"],["mean"])
ep_feat.set_n_jobs(7)
df = ep_feat.featurize_dataframe(df, col_id="composition")

# Selecionando as colunas que contém informações sobre condutividade térmica



c:\Users\guilherme220044\Anaconda3\envs\IP_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ElementProperty:  18%|█▊        | 1640/8924 [05:20<20:15,  5.99it/s]

In [ ]:
df = df.dropna(axis=1) # retirando as linhas que possuirem algum dado faltante (NaN, Not a Number)

In [ ]:
from matminer.featurizers.structure import DensityFeatures

densityf = DensityFeatures()

densityf.feature_labels() # utilizado para mostrar os dados disponiveis com esse metodo.

['density', 'vpa', 'packing fraction']

In [ ]:
df = densityf.featurize_dataframe(df, "structure", True, True)

DensityFeatures: 100%|██████████| 8924/8924 [38:03<00:00,  3.91it/s]   


In [ ]:
df = df.rename(columns={"PymatgenData mean melting_point":"melting_point"})
df = df.rename(columns={"PymatgenData mean thermal_conductivity":"thermal_conductivity"})

In [ ]:
df.describe()

,pf_n,pf_p,formula,structure,composition,melting_point,thermal_conductivity,density,vpa,packing fraction
1167,0.542,0.0549,H2,[[-1.95019892e-08 2.25189574e+00 1.58892283e...,(H),14.01,0.1805,0.039976,41.867906,0.001563
4397,0.694,0.1910,H2,[[-2.56226548e-06 2.96249947e+00 9.52972900e...,(H),14.01,0.1805,0.038513,43.458888,0.001506
8656,1.220,0.3600,H2,[[-4.88184487e-18 4.88184487e-18 3.75064620e...,(H),14.01,0.1805,0.146850,11.397498,0.005742


In [ ]:
quantidade = len(set(df["formula"]))

print(f"Com esses dados temos {quantidade} moleculas distintas.")

Com esses dados temos 7380 moleculas distintas.
